In [30]:
import numpy as np
import pandas as pd
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn import Sequential
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [31]:
# basic_transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Resize((224,224))
# ])

# temp_train_dataset = datasets.OxfordIIITPet(root='./dataF',split='trainval',target_types='category',transform=basic_transform,download=True)

# loader = DataLoader(dataset=temp_train_dataset,batch_size=64,shuffle=False,num_workers=2)

# mean,std = 0.0,0.0
# total_number_samples = 0

# for img,_ in tqdm(loader):
#     batch_size = img.size(0)
#     img = img.view(batch_size,img.size(1),-1)
#     mean = img.mean(2).sum(0)
#     std = img.std(2).sum(0)
#     total_number_samples += batch_size

# mean /= total_number_samples
# std /= total_number_samples

# print("Mean:", mean)
# print("std:", std)


In [32]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomCrop(224,padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.OxfordIIITPet(root='./dataF',split='trainval',target_types='category',transform=train_transform,download=True)
test_dataset = datasets.OxfordIIITPet(root='./dataF',split='test',target_types='category',transform=test_transform,download=True)

train_loader = DataLoader(train_dataset,batch_size=64,shuffle=True,num_workers=2)
test_loader = DataLoader(test_dataset,batch_size=64,shuffle=False,num_workers=2)

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

alexNet = models.alexnet(pretrained=True).to(device)
for param in alexNet.parameters():
    param.requires_grad = False

feature_extractor = Sequential(*list(alexNet.classifier.children())[:-1])

/home/shwetank/Desktop/ShwetankAssignmentMT2025725/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/shwetank/Desktop/ShwetankAssignmentMT2025725/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [34]:
def extract_feature(loader):
    features,labels = [],[]
    alexNet.eval()
    with torch.no_grad():
        for imgs,lbls in tqdm(loader):
            imgs = imgs.to(device)
            x = alexNet.features(imgs)
            x = alexNet.avgpool(x)
            x = torch.flatten(x,1)
            x = feature_extractor(x)
            features.append(x.cpu())
            labels.append(lbls.cpu())
        return torch.cat(features),torch.cat(labels)
    
X_train, y_train = extract_feature(train_loader)
X_test, y_test = extract_feature(test_loader)

X_train

100%|██████████| 58/58 [00:12<00:00,  4.63it/s]


tensor([[0.0000, 0.0000, 0.2064,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 3.0781, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [35]:
X_train = X_train.numpy()
y_train = y_train.numpy()
X_test = X_test.numpy()
y_test = y_test.numpy()

##Logistic Regression##
clf_logreg = LogisticRegression(max_iter=1000)
clf_logreg.fit(X_train,y_train)
pred_logreg = clf_logreg.predict(X_test)
acc_logreg = accuracy_score(y_test, pred_logreg)
print("Logistic Regression Accuracy:", acc_logreg)

Logistic Regression Accuracy: 0.7454347233578632


In [36]:
clf_rf = RandomForestClassifier(n_estimators=100, random_state=42)
clf_rf.fit(X_train, y_train)
pred_rf = clf_rf.predict(X_test)
acc_rf = accuracy_score(y_test, pred_rf)
print("Random Forest Accuracy:", acc_rf)

Random Forest Accuracy: 0.7239029708367403
